### Exploring The Impact of Optimizers and Activation Functions On OODN 

In [82]:
from os import listdir

import time
import json

import torch

from torchvision.datasets import mnist, FashionMNIST
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch.optim import SGD, Adam
from torch.nn import Module
from torch import nn
from torch.nn import CrossEntropyLoss
from torchvision.models.resnet import Bottleneck, ResNet

from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms

import numpy as np
import pandas as pd

from openood.evaluators import metrics

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


### Supported Activation Functions

For activation functions, we are considering ReLU, Softplus, Swish. *Note that we may conduct experiments for a subset based on the compute resources available*

In [3]:
def get_activation_fn(activation):
    if activation == 'relu':
        return nn.ReLU()
    elif activation == 'softplus':
        return nn.Softplus()
    elif activation == 'swish':
        return nn.Swish()
    return None

### Supported Networks

Currently, we support LeNet and ResNet50.

### LeNet

In [4]:
class LeNet(nn.Module):
    def __init__(self, num_classes, num_channel=3, activation='relu'):
        super(LeNet, self).__init__()
        self.num_classes = num_classes
        self.feature_size = 84
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=num_channel,
                      out_channels=6,
                      kernel_size=5,
                      stride=1,
                      padding=2), get_activation_fn(activation), nn.MaxPool2d(kernel_size=2))

        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
             get_activation_fn(activation), nn.MaxPool2d(kernel_size=2))

        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels=16,
                      out_channels=120,
                      kernel_size=5,
                      stride=1), get_activation_fn(activation))

        self.classifier1 = nn.Linear(in_features=120, out_features=84)
        self.relu = get_activation_fn(activation)
        self.fc = nn.Linear(in_features=84, out_features=num_classes)

    def get_fc(self):
        fc = self.fc
        return fc.weight.cpu().detach().numpy(), fc.bias.cpu().detach().numpy()

    def forward(self, x, return_feature=False, return_feature_list=False):
        feature1 = self.block1(x)
        feature2 = self.block2(feature1)
        feature3 = self.block3(feature2)
        feature3 = feature3.view(feature3.shape[0], -1)
        feature = self.relu(self.classifier1(feature3))
        logits_cls = self.fc(feature)
        feature_list = [feature1, feature2, feature3, feature]
        if return_feature:
            return logits_cls, feature
        elif return_feature_list:
            return logits_cls, feature_list
        else:
            return logits_cls

    def forward_threshold(self, x, threshold):
        feature1 = self.block1(x)
        feature2 = self.block2(feature1)
        feature3 = self.block3(feature2)
        feature3 = feature3.view(feature3.shape[0], -1)
        feature = self.relu(self.classifier1(feature3))
        feature = feature.clip(max=threshold)
        logits_cls = self.fc(feature)

        return logits_cls

### ResNet50

In [5]:
class ResNet50(ResNet):
    def __init__(self,
                 block=Bottleneck,
                 layers=[3, 4, 6, 3],
                 num_classes=1000):
        super(ResNet50, self).__init__(block=block,
                                       layers=layers,
                                       num_classes=num_classes)
        self.feature_size = 2048


    def forward(self, x, return_feature=False, return_feature_list=False):
        feature1 = self.relu(self.bn1(self.conv1(x)))
        feature1 = self.maxpool(feature1)
        feature2 = self.layer1(feature1)
        feature3 = self.layer2(feature2)
        feature4 = self.layer3(feature3)
        feature5 = self.layer4(feature4)
        feature5 = self.avgpool(feature5)
        feature = feature5.view(feature5.size(0), -1)
        logits_cls = self.fc(feature)

        feature_list = [feature1, feature2, feature3, feature4, feature5]
        if return_feature:
            return logits_cls, feature
        elif return_feature_list:
            return logits_cls, feature_list
        else:
            return logits_cls

    def forward_threshold(self, x, threshold):
        feature1 = self.relu(self.bn1(self.conv1(x)))
        feature1 = self.maxpool(feature1)
        feature2 = self.layer1(feature1)
        feature3 = self.layer2(feature2)
        feature4 = self.layer3(feature3)
        feature5 = self.layer4(feature4)
        feature5 = self.avgpool(feature5)
        feature = feature5.clip(max=threshold)
        feature = feature.view(feature.size(0), -1)
        logits_cls = self.fc(feature)

        return logits_cls

    def get_fc(self):
        fc = self.fc
        return fc.weight.cpu().detach().numpy(), fc.bias.cpu().detach().numpy()
    
def get_resnet_model(activation_function_type, n_classes):
    resnet_model = ResNet50(num_classes=n_classes)
    resnet_model.to(device)    
    return resnet_model

def set_activation_function(resnet_model, activation_function_type):
    resnet_model.relu = get_activation_fn(activation_function_type)
    resnet_model.layer1[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer1[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer1[2].relu = get_activation_fn(activation_function_type)

    resnet_model.layer2[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer2[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer2[2].relu = get_activation_fn(activation_function_type)
    resnet_model.layer2[3].relu = get_activation_fn(activation_function_type)

    resnet_model.layer3[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[2].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[3].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[4].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[5].relu = get_activation_fn(activation_function_type)


    resnet_model.layer4[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer4[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer4[2].relu = get_activation_fn(activation_function_type)
    
    return resnet_model

In [6]:
def get_model(config):
    activation_function_type = config["activation_function_type"]
    network_type = config["network"]
    n_classes = config["n_classes"]
    
    if network_type == "lenet":
        model =  LeNet(num_classes=n_classes, num_channel=1, activation=activation_function_type)
    elif network_type == "resnet50":
        model = get_resnet_model(activation_function_type, n_classes)
    else:
        raise Exception("Currently we only support lenet or resnet50")
    
    return model
    

### Supported Post-Hoc OODN Processors

#### The first post processor we consider is ODIN

In [32]:
class ODINPostprocessor():
    def __init__(self, temperature, noise):
        self.temperature = temperature
        self.noise = noise
        
    def postprocess(self, net: nn.Module, data):
        net.eval()
        data.requires_grad = True
        output = net(data)

        # Calculating the perturbation we need to add, that is,
        # the sign of gradient of cross entropy loss w.r.t. input
        criterion = nn.CrossEntropyLoss()

        labels = output.detach().argmax(axis=1)

        # Using temperature scaling
        output = output / self.temperature

        loss = criterion(output, labels)
        loss.backward()

        # Normalizing the gradient to binary in {0, 1}
        gradient = torch.ge(data.grad.detach(), 0)
        gradient = (gradient.float() - 0.5) * 2

        # Scaling values taken from original code       
        gradient[:, 0] = (gradient[:, 0]) / (63.0 / 255.0)
        if gradient.shape[1] == 3:
            gradient[:, 1] = (gradient[:, 1]) / (62.1 / 255.0)
            gradient[:, 2] = (gradient[:, 2]) / (66.7 / 255.0)

        # Adding small perturbations to images
        tempInputs = torch.add(data.detach(), gradient, alpha=-self.noise)
        output = net(tempInputs)
        output = output / self.temperature

        # Calculating the confidence after adding perturbations
        nnOutput = output.detach()
        nnOutput = nnOutput - nnOutput.max(dim=1, keepdims=True).values
        nnOutput = nnOutput.exp() / nnOutput.exp().sum(dim=1, keepdims=True)

        conf, pred = nnOutput.max(dim=1)

        return pred, conf
    
    def inference(self, net: nn.Module, data_loader: DataLoader):
        pred_list, conf_list, label_list = [], [], []
        for idx, loaded_data in enumerate(data_loader):
            data, label = loaded_data[0], loaded_data[1]
            if idx % 50 == 0:
                print(f'Performing inference on batch: {idx}')
            pred, conf = self.postprocess(net, data.to(device))
            for idx in range(len(data)):
                pred_list.append(pred[idx].tolist())
                conf_list.append(conf[idx].tolist())
                label_list.append(label[idx].tolist())

        # convert values into numpy array
        pred_list = np.array(pred_list, dtype=int)
        conf_list = np.array(conf_list)
        label_list = np.array(label_list, dtype=int)

        return pred_list, conf_list, label_list

#### We consider the Maximum Classifier Discrepancy Post OODN method

https://arxiv.org/pdf/1712.02560.pdf

In [84]:
class MCDPostprocessor():
    @torch.no_grad()
    def postprocess(self, net: nn.Module, data):
        logits1, logits2 = net(data, return_double=True)
        score1 = torch.softmax(logits1, dim=1)
        score2 = torch.softmax(logits2, dim=1)
        conf = -torch.sum(torch.abs(score1 - score2), dim=1)
        _, pred = torch.max(score1, dim=1)
        return pred, conf
    
    def inference(self, net: nn.Module, data_loader: DataLoader):
        pred_list, conf_list, label_list = [], [], []
        for idx, loaded_data in enumerate(data_loader):
            data, label = loaded_data[0], loaded_data[1]
            if idx % 50 == 0:
                print(f'Performing inference on batch: {idx}')
            pred, conf = self.postprocess(net, data.to(device))
            for idx in range(len(data)):
                pred_list.append(pred[idx].tolist())
                conf_list.append(conf[idx].tolist())
                label_list.append(label[idx].tolist())

        # convert values into numpy array
        pred_list = np.array(pred_list, dtype=int)
        conf_list = np.array(conf_list)
        label_list = np.array(label_list, dtype=int)

        return pred_list, conf_list, label_list

In [85]:
def get_postprocessor(postprocessor_type="odin"):
    if postprocessor_type == "odin":
        postprocessor = ODINPostprocessor(1000, 0.0014)
    elif postprocessor_type == "mcd":
        postprocessor = MCDPostprocessor()
    return postprocessor

### Supported Out of Distribution Detection Metrics

What metrics do we specifically care about here?

**FPR@95** measures the false positive rate (FPR) when the true positive rate (TPR) is
equal to 95%. Lower scores indicate better performance. 

**AUROC** measures the area under the
Receiver Operating Characteristic (ROC) curve, which displays the relationship between TPR and
FPR. The area under the ROC curve can be interpreted as the probability that a positive ID example
will have a higher detection score than a negative OOD example. 

**AUPR** measures the area under
the Precision-Recall (PR) curve. The PR curve is created by plotting precision versus recall. Similar
to AUROC, we consider ID samples as positive, so that the score corresponds to the AUPR-In metric
in some works

In [81]:
def calculate_oodn_metrics(model, postprocessor_type, id_test_loader, ood_test_loader, ood_name):
    postprocessor = get_postprocessor(postprocessor_type)
    id_pred, id_conf, id_gt = postprocessor.inference(
                model, id_test_loader)

    ood_pred, ood_conf, ood_gt = postprocessor.inference(
        model, ood_test_loader)

    ood_gt = -1 * np.ones_like(ood_gt)  # hard set to -1 as ood
    pred = np.concatenate([id_pred, ood_pred])
    conf = np.concatenate([id_conf, ood_conf])
    label = np.concatenate([id_gt, ood_gt])
    ood_metrics = metrics.compute_all_metrics(conf, label, pred)

    return print_and_get_formatted_metrics(ood_metrics, ood_name)

def print_and_get_formatted_metrics(metrics, dataset_name):
    [fpr, auroc, aupr_in, aupr_out,
     ccr_4, ccr_3, ccr_2, ccr_1, accuracy] \
     = metrics

    write_content = {
        'dataset': dataset_name,
        'FPR@95': '{:.2f}'.format(100 * fpr),
        'AUROC': '{:.2f}'.format(100 * auroc),
        'AUPR_IN': '{:.2f}'.format(100 * aupr_in),
        'AUPR_OUT': '{:.2f}'.format(100 * aupr_out),
        'CCR_4': '{:.2f}'.format(100 * ccr_4),
        'CCR_3': '{:.2f}'.format(100 * ccr_3),
        'CCR_2': '{:.2f}'.format(100 * ccr_2),
        'CCR_1': '{:.2f}'.format(100 * ccr_1),
        'ACC': '{:.2f}'.format(100 * accuracy)
    }

    fieldnames = list(write_content.keys())

    # print ood metric results
    print('FPR@95: {:.2f}, AUROC: {:.2f}'.format(100 * fpr, 100 * auroc),
          end=' ',
          flush=True)
    print('AUPR_IN: {:.2f}, AUPR_OUT: {:.2f}'.format(
        100 * aupr_in, 100 * aupr_out),
          flush=True)
    print('CCR: {:.2f}, {:.2f}, {:.2f}, {:.2f},'.format(
        ccr_4 * 100, ccr_3 * 100, ccr_2 * 100, ccr_1 * 100),
          end=' ',
          flush=True)
    print('ACC: {:.2f}'.format(accuracy * 100), flush=True)
    print(u'\u2500' * 70, flush=True)
    return write_content

def load_results_into_df(dir_path):
    res_files = [dir_path+each for each in listdir(dir_path)]
    all_results = []
    columns = ['optimizer_type', 'activation_function_type', 'postprocessor_type', 'trial', 'AUROC']
    for fp in res_files:
        f = open(fp)
        data = json.load(f)
        for trial, results in data.items():
            all_results.append([
                    results['optimizer_type'],
                    results['activation_function_type'],
                    results['postprocessor_type'],
                    trial,
                    float(results['AUROC'])
                ])
    df = pd.DataFrame(all_results, columns=columns)
    return df

In [30]:
def get_optimizer(model, config):
    params = model.parameters()
    lr = config['lr']
    momentum = config['momentum']
    weight_decay = config['weight_decay']
    optimizer_type = config['optimizer_type']
    
    print(f'Getting optimizer for type: {optimizer_type}...')
    if optimizer_type == 'SGD':
        return SGD(params, 
              lr=lr, 
              momentum=momentum,
              weight_decay=weight_decay)
    elif optimizer_type == 'Adam':
        return Adam(params, 
                    lr=lr, 
                    weight_decay=weight_decay)
    else:
        raise Exception("Invalid optimizer_type provided, only SGD and Adam are supported currently")

def get_wilds_loader(dataset, split, batch_size):
    d = dataset.get_subset(
        "train",
        transform=transforms.Compose(
            [transforms.Resize((448, 448)), transforms.ToTensor()]
        ),
    )
    # Prepare the standard data loader
    return get_train_loader("standard", d, batch_size=batch_size)

def get_data_loaders(config):
    data_loaders = {}
    dataset_name = config["dataset_name"]
    dataset_type = config["dataset_type"]
    batch_size = config['batch_size']
    
    wilds_id_test_split = "id_val" if dataset_name == "camelyon17" else "id_test"
    if dataset_type == "wilds":
        # wilds dataset
        dataset = get_dataset(dataset=dataset_name, download=True)
        data_loaders["train"] = get_wilds_loader(dataset, "train", batch_size)
        data_loaders["ood_test"] = get_wilds_loader(dataset, "test", batch_size)
        data_loaders["id_test"] = get_wilds_loader(dataset, wilds_id_test_split, batch_size)
    elif dataset_name == "mnist":
        # mnist dataset
        train_dataset = mnist.MNIST(root='data', download=True, train=True, transform=ToTensor())
        test_dataset = mnist.MNIST(root='data', download=True, train=False, transform=ToTensor())
        fashion_test_dataset = mnist.FashionMNIST(root='data', download=True,train=False,transform=ToTensor())

        data_loaders["train"] = DataLoader(train_dataset, batch_size=batch_size)
        data_loaders["id_test"] = DataLoader(test_dataset, batch_size=batch_size)
        data_loaders["ood_test"] = DataLoader(fashion_test_dataset, batch_size=batch_size)
    
    return data_loaders

In [52]:
def train_resnet_model_given_opti_activation_fn(config):
    # get the train loader
    train_loader = config["data_loaders"]["train"]
    
    # get the resnet model with the replaced activation functions
    model = get_model(config)
    model.to(device)
    
    # get the optimizer
    sgd = get_optimizer(model, config)
    
    loss_fn = CrossEntropyLoss()

    for current_epoch in range(config['epochs']):
        model.train()
        print('Training epoch: {}'.format(current_epoch))
        for idx, (loader_data) in enumerate(train_loader):
            train_x, train_label = loader_data[0].to(device), loader_data[1].to(device)
            sgd.zero_grad()
            predict_y = model(train_x.float())
            loss = loss_fn(predict_y, train_label.long())
            if idx % 100 == 0:
                print('idx: {}, loss: {}'.format(idx, loss.sum().item()))
            loss.backward()
            sgd.step()
    
    torch.save(model, config['model_name'])
    return model

def run_full_oodn_pipeline(config):
    metrics = {}
    for i in range(config["trials"]):
        model_name = f"models/{config['network']}_{config['dataset_name']}_{config['activation_function_type']}_{config['optimizer_type']}_{i}.pkl"
        print(f'Running model: {model_name}...')
        config['model_name'] = model_name
        # train model
        model = train_resnet_model_given_opti_activation_fn(config)
        # calculate oodn metrics
        metrics[i] = calculate_oodn_metrics(model,
                               "odin", 
                               config["data_loaders"]["id_test"], 
                               config["data_loaders"]["ood_test"], 
                               config["dataset_name"])
        metrics[i]['optimizer_type'] = config['optimizer_type']
        metrics[i]['activation_function_type'] = config['activation_function_type']
        metrics[i]['postprocessor_type'] = config['postprocessor_type']
        
    experiment_name = f"{config['results_dir']}/{config['dataset_name']}_{config['network']}_{config['postprocessor_type']}_{config['activation_function_type']}_{config['optimizer_type']}.json"
    with open(experiment_name, 'w') as fp:
        json.dump(metrics, fp)
    return metrics

### Study 1: LeNet5, MNIST as the ID Dataset, FashionMNIST as the OOD Dataset

#### Study 1(a): Combination: SGD + ReLU

In [54]:
config_1a = {
    "batch_size": 128,
    "n_classes": 10,
    "dataset_name": "mnist",
    "epochs": 10,
    "version": time.time(),
    "lr": 0.1,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "SGD",
    "activation_function_type": "relu",
    "network": "lenet",
    "postprocessor_type": "odin",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1a["data_loaders"] = get_data_loaders(config_1a)
run_full_oodn_pipeline(config_1a)

Running model: models/lenet_mnist_relu_SGD_0.pkl...
Getting optimizer for type: SGD...
Training epoch: 0
idx: 0, loss: 2.300678014755249
idx: 100, loss: 0.23158413171768188
idx: 200, loss: 0.3385574519634247
idx: 300, loss: 0.14093522727489471
idx: 400, loss: 0.3233451247215271
Training epoch: 1
idx: 0, loss: 0.11926180869340897
idx: 100, loss: 0.12861403822898865
idx: 200, loss: 0.16344964504241943
idx: 300, loss: 0.06893038004636765
idx: 400, loss: 0.2226434201002121
Training epoch: 2
idx: 0, loss: 0.18933868408203125
idx: 100, loss: 0.1398666501045227
idx: 200, loss: 0.08190067112445831
idx: 300, loss: 0.05670902132987976
idx: 400, loss: 0.2548925280570984
Training epoch: 3
idx: 0, loss: 0.17106886208057404
idx: 100, loss: 0.0651470348238945
idx: 200, loss: 0.09267112612724304
idx: 300, loss: 0.08352600038051605
idx: 400, loss: 0.2009848803281784
Training epoch: 4
idx: 0, loss: 0.07830512523651123
idx: 100, loss: 0.05214201658964157
idx: 200, loss: 0.06437063962221146
idx: 300, loss

{0: {'dataset': 'mnist',
  'FPR@95': '0.95',
  'AUROC': '99.60',
  'AUPR_IN': '99.64',
  'AUPR_OUT': '99.56',
  'CCR_4': '58.68',
  'CCR_3': '85.04',
  'CCR_2': '94.07',
  'CCR_1': '97.25',
  'ACC': '97.89',
  'optimizer_type': 'SGD',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin'},
 1: {'dataset': 'mnist',
  'FPR@95': '2.89',
  'AUROC': '99.36',
  'AUPR_IN': '99.40',
  'AUPR_OUT': '99.35',
  'CCR_4': '53.81',
  'CCR_3': '69.97',
  'CCR_2': '89.37',
  'CCR_1': '96.98',
  'ACC': '97.97',
  'optimizer_type': 'SGD',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin'},
 2: {'dataset': 'mnist',
  'FPR@95': '0.22',
  'AUROC': '99.81',
  'AUPR_IN': '99.83',
  'AUPR_OUT': '99.80',
  'CCR_4': '80.52',
  'CCR_3': '92.84',
  'CCR_2': '96.66',
  'CCR_1': '98.34',
  'ACC': '98.64',
  'optimizer_type': 'SGD',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin'}}

#### 1 (b) SGD + SoftPlus

In [58]:
config_1b = {
    "batch_size": 128,
    "dataset_name": "mnist",
    "n_classes": 10,
    "epochs": 10,
    "version": time.time(),
    "lr": 0.1,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "SGD",
    "activation_function_type": "softplus",
    "network": "lenet",
    "postprocessor_type": "odin",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1b["data_loaders"] = get_data_loaders(config_1b)
run_full_oodn_pipeline(config_1b)

Running model: models/lenet_mnist_softplus_SGD_0.pkl...
Getting optimizer for type: SGD...
Training epoch: 0
idx: 0, loss: 2.4468095302581787
idx: 100, loss: 2.3068735599517822
idx: 200, loss: 2.3037972450256348
idx: 300, loss: 2.013766050338745
idx: 400, loss: 0.9880338907241821
Training epoch: 1
idx: 0, loss: 0.47247204184532166
idx: 100, loss: 0.4614843428134918
idx: 200, loss: 0.4421844780445099
idx: 300, loss: 0.2332533895969391
idx: 400, loss: 0.5398097038269043
Training epoch: 2
idx: 0, loss: 0.12531839311122894
idx: 100, loss: 0.1766596883535385
idx: 200, loss: 0.19630563259124756
idx: 300, loss: 0.14620856940746307
idx: 400, loss: 0.2683812975883484
Training epoch: 3
idx: 0, loss: 0.10990933328866959
idx: 100, loss: 0.12922294437885284
idx: 200, loss: 0.12120915949344635
idx: 300, loss: 0.08533128350973129
idx: 400, loss: 0.23656682670116425
Training epoch: 4
idx: 0, loss: 0.09355681389570236
idx: 100, loss: 0.1110059916973114
idx: 200, loss: 0.1473049521446228
idx: 300, loss:

{0: {'dataset': 'mnist',
  'FPR@95': '3.61',
  'AUROC': '98.99',
  'AUPR_IN': '99.18',
  'AUPR_OUT': '98.70',
  'CCR_4': '45.28',
  'CCR_3': '81.40',
  'CCR_2': '89.68',
  'CCR_1': '96.18',
  'ACC': '97.97',
  'optimizer_type': 'SGD',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin'},
 1: {'dataset': 'mnist',
  'FPR@95': '3.05',
  'AUROC': '99.07',
  'AUPR_IN': '99.19',
  'AUPR_OUT': '98.90',
  'CCR_4': '6.62',
  'CCR_3': '72.95',
  'CCR_2': '89.86',
  'CCR_1': '96.13',
  'ACC': '97.66',
  'optimizer_type': 'SGD',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin'},
 2: {'dataset': 'mnist',
  'FPR@95': '4.31',
  'AUROC': '98.80',
  'AUPR_IN': '99.05',
  'AUPR_OUT': '98.31',
  'CCR_4': '55.94',
  'CCR_3': '78.45',
  'CCR_2': '88.74',
  'CCR_1': '95.57',
  'ACC': '97.51',
  'optimizer_type': 'SGD',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin'}}

#### 1(c) Adam + ReLU

In [56]:
config_1c = {
    "batch_size": 128,
    "dataset_name": "mnist",
    "n_classes": 10,
    "epochs": 10,
    "version": time.time(),
    "lr": 0.01,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "Adam",
    "activation_function_type": "relu",
    "network": "lenet",
    "postprocessor_type": "odin",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1c["data_loaders"] = get_data_loaders(config_1c)
run_full_oodn_pipeline(config_1c)

Running model: models/lenet_mnist_relu_Adam_0.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.301894187927246
idx: 100, loss: 0.11984912306070328
idx: 200, loss: 0.15497374534606934
idx: 300, loss: 0.06791342049837112
idx: 400, loss: 0.22912515699863434
Training epoch: 1
idx: 0, loss: 0.07157646864652634
idx: 100, loss: 0.05058322101831436
idx: 200, loss: 0.09429037570953369
idx: 300, loss: 0.08827465027570724
idx: 400, loss: 0.1425236165523529
Training epoch: 2
idx: 0, loss: 0.08118437230587006
idx: 100, loss: 0.029627040028572083
idx: 200, loss: 0.11240573972463608
idx: 300, loss: 0.049749165773391724
idx: 400, loss: 0.10378781706094742
Training epoch: 3
idx: 0, loss: 0.05978626012802124
idx: 100, loss: 0.04203808680176735
idx: 200, loss: 0.12809190154075623
idx: 300, loss: 0.072877898812294
idx: 400, loss: 0.12331315875053406
Training epoch: 4
idx: 0, loss: 0.04495770111680031
idx: 100, loss: 0.02584172785282135
idx: 200, loss: 0.11872470378875732
idx: 3

{0: {'dataset': 'mnist',
  'FPR@95': '2.70',
  'AUROC': '99.40',
  'AUPR_IN': '99.44',
  'AUPR_OUT': '99.38',
  'CCR_4': '52.68',
  'CCR_3': '74.63',
  'CCR_2': '91.00',
  'CCR_1': '97.29',
  'ACC': '98.42',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin'},
 1: {'dataset': 'mnist',
  'FPR@95': '2.21',
  'AUROC': '99.41',
  'AUPR_IN': '99.47',
  'AUPR_OUT': '99.36',
  'CCR_4': '73.34',
  'CCR_3': '81.76',
  'CCR_2': '91.29',
  'CCR_1': '97.00',
  'ACC': '97.89',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin'},
 2: {'dataset': 'mnist',
  'FPR@95': '0.97',
  'AUROC': '99.55',
  'AUPR_IN': '99.61',
  'AUPR_OUT': '99.51',
  'CCR_4': '73.59',
  'CCR_3': '89.21',
  'CCR_2': '94.54',
  'CCR_1': '97.55',
  'ACC': '98.29',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin'}}

#### 1(d) Adam + Softplus

In [57]:
config_1d = {
    "batch_size": 128,
    "dataset_name": "mnist",
    "n_classes": 10,
    "epochs": 10,
    "version": time.time(),
    "lr": 0.01,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "Adam",
    "activation_function_type": "softplus",
    "network": "lenet",
    "postprocessor_type": "odin",
    "dataset_type": "mnist",
    "trials": 3,
    "results_dir": "mnist-study"
}
config_1d["data_loaders"] = get_data_loaders(config_1d)
run_full_oodn_pipeline(config_1d)

Running model: models/lenet_mnist_softplus_Adam_0.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.3255388736724854
idx: 100, loss: 2.3116369247436523
idx: 200, loss: 2.305769443511963
idx: 300, loss: 2.2983803749084473
idx: 400, loss: 2.307865619659424
Training epoch: 1
idx: 0, loss: 2.276770830154419
idx: 100, loss: 0.8078237175941467
idx: 200, loss: 0.25390756130218506
idx: 300, loss: 0.32934701442718506
idx: 400, loss: 0.3490292727947235
Training epoch: 2
idx: 0, loss: 0.1403365433216095
idx: 100, loss: 0.2735982835292816
idx: 200, loss: 0.14142420887947083
idx: 300, loss: 0.18107478320598602
idx: 400, loss: 0.2841825485229492
Training epoch: 3
idx: 0, loss: 0.10381539165973663
idx: 100, loss: 0.18071672320365906
idx: 200, loss: 0.11766538769006729
idx: 300, loss: 0.14970530569553375
idx: 400, loss: 0.25166428089141846
Training epoch: 4
idx: 0, loss: 0.13393616676330566
idx: 100, loss: 0.14538753032684326
idx: 200, loss: 0.12768563628196716
idx: 300, los

{0: {'dataset': 'mnist',
  'FPR@95': '30.70',
  'AUROC': '93.93',
  'AUPR_IN': '94.30',
  'AUPR_OUT': '93.41',
  'CCR_4': '3.42',
  'CCR_3': '20.72',
  'CCR_2': '50.00',
  'CCR_1': '81.76',
  'ACC': '95.28',
  'optimizer_type': 'Adam',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin'},
 1: {'dataset': 'mnist',
  'FPR@95': '5.51',
  'AUROC': '98.81',
  'AUPR_IN': '98.96',
  'AUPR_OUT': '98.67',
  'CCR_4': '43.70',
  'CCR_3': '68.11',
  'CCR_2': '86.62',
  'CCR_1': '95.13',
  'ACC': '97.47',
  'optimizer_type': 'Adam',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin'},
 2: {'dataset': 'mnist',
  'FPR@95': '39.54',
  'AUROC': '93.00',
  'AUPR_IN': '93.58',
  'AUPR_OUT': '92.81',
  'CCR_4': '2.51',
  'CCR_3': '19.59',
  'CCR_2': '49.86',
  'CCR_1': '78.18',
  'ACC': '94.28',
  'optimizer_type': 'Adam',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin'}}

### Robustness Analysis For Study 1

In [86]:
df_study_1 = load_results_into_df('mnist-study/')
df_study_1

,optimizer_type,activation_function_type,postprocessor_type,trial,AUROC
0,SGD,softplus,odin,0,98.99
1,SGD,softplus,odin,1,99.07
2,SGD,softplus,odin,2,98.80
3,Adam,softplus,odin,0,93.93
4,Adam,softplus,odin,1,98.81
5,Adam,softplus,odin,2,93.00
6,SGD,relu,odin,0,99.60
7,SGD,relu,odin,1,99.36
8,SGD,relu,odin,2,99.81
9,Adam,relu,odin,0,99.40


In [78]:
df_study_1.groupby(['optimizer_type', 'activation_function_type'])['AUROC'].describe()

count       mean       std    min  \
optimizer_type activation_function_type                                      
Adam           relu                        3.0  99.453333  0.083865  99.40   
               softplus                    3.0  95.246667  3.120774  93.00   
SGD            relu                        3.0  99.590000  0.225167  99.36   
               softplus                    3.0  98.953333  0.138684  98.80   

                                            25%    50%     75%    max  
optimizer_type activation_function_type                                
Adam           relu                      99.405  99.41  99.480  99.55  
               softplus                  93.465  93.93  96.370  98.81  
SGD            relu                      99.480  99.60  99.705  99.81  
               softplus                  98.895  98.99  99.030  99.07

In [79]:
df_study_1.groupby(['activation_function_type'])['AUROC'].describe()

,count,mean,std,min,25%,50%,75%,max
activation_function_type,,,,,,,,
relu,6.0,99.521667,0.169401,99.36,99.4025,99.480,99.5875,99.81
softplus,6.0,97.100000,2.832878,93.00,95.1475,98.805,98.9450,99.07


In [80]:
df_study_1.groupby(['optimizer_type'])['AUROC'].describe()

,count,mean,std,min,25%,50%,75%,max
optimizer_type,,,,,,,,
Adam,6.0,97.350000,3.034357,93.0,95.15,99.105,99.4075,99.55
SGD,6.0,99.271667,0.386751,98.8,99.01,99.215,99.5400,99.81


### Study 2: Resnet50, WILDS iwildcam as the Dataset

In [20]:
config_2a = {
    "batch_size": 16,
    "n_classes": 2,
    "dataset_name": "camelyon17",
    "epochs": 10,
    "version": time.time(),
    "lr": 0.01,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "SGD",
    "activation_function_type": "softplus",
    "network": "resnet50",
    "postprocessor_type": "odin",
    "dataset_type": "wilds"
}
config_2a["data_loaders"] = get_data_loaders(config_2a)
# run_full_oodn_pipeline(config_1d)